In [7]:
#=================================================================================================================
#                                           Libraries
#=================================================================================================================

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import PIL 
import os

In [38]:
#=================================================================================================================
#                                           Image Preparation
#=================================================================================================================
from sklearn.model_selection import train_test_split

def load_custom_dataset(dataset_path):
    images = []
    labels = []

    for root, dirs, files in os.walk(dataset_path):
        for file in files:
            if file.endswith(('.png', '.jpg', '.jpeg')):
                label = os.path.basename(root)
                image_path = os.path.join(root, file)

                images.append(preprocess_image(image_path))
#                 images.append(image_path)
                labels.append(label)
            
    return images, labels

def preprocess_image(image_path):
    img = PIL.Image.open(image_path)
            
    # convert to grayscale
    img = img.convert('L')
    # resize 
    img = img.resize((8,8))
    
    return np.array(img).flatten()



In [39]:
#=================================================================================================================
#                                          Loading Dataset
#=================================================================================================================

images, labels = load_custom_dataset("Dataset")

test_size = 1/5
random_state = 42
images_train, images_test, labels_train, labels_test = train_test_split(images, 
                                                                        labels, 
                                                                        test_size=test_size, 
                                                                        random_state=random_state)

In [42]:
#=================================================================================================================
#                                           Implement Your Algorithm
#=================================================================================================================



In [8]:
#=================================================================================================================
#                                           NEURAL NETWORK (BONUS)
#=================================================================================================================

#  Write You Code Here

In [9]:
#=================================================================================================================
#                                           Test Section
#=================================================================================================================


# Leave it blank

In [ ]:
#=================================================================================================================
#                                           Visualization
#=================================================================================================================

#  Write You Code Here